## Loading the necessary models for the training purpose

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers


## Loading the Quantized model from Mistral AI


In [10]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"  # THis guy has many quantized models
model = AutoModelForCausalLM.from_pretrained( # using pretrained method we can import modle
            model_name,   
            device_map = "auto", # Optimum way to split the loading cpu and gpu working
            trust_remote_code=False, #not allow custom model file to run on local machine while downloading
            revision='main' # we want the main version

)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinci\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.2-GPTQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
CUDA extension not installed.
CUDA extension not installed.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\transformers\modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.b

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Loading the tokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Testing the loaded model

In [12]:
model.eval() # in evaluation model, backprop and dropouts are deactivated

## Prompting
comment = "Great content, thank you"
prompt = f'''[INST] {comment} [/INST]''' # Instruction tokens for the model


inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:674: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<s> [INST] Great content, thank you [/INST] I'm glad you found the content helpful. If you have any specific questions or topics you'd like me to cover in the future, please don't hesitate to ask. I'm here to help!

In the meantime, feel free to explore some of the other articles and resources on this website. I've written about a wide range of topics related to personal finance, investing, and entrepreneurship. And if you have any suggestions for future content, I'd love to hear them.

Thanks again for reading, and I wish you all the best on your financial journey!</s>


## Prompt Engineering for the Loaded model

In [26]:
comment = "Shut Up!"
instruction_string = f"""Determine if the following sentence contains abusive language. If it does, specify the level of abusiveness as Low, Medium, or High. \n
    
    Definitions: \n
    - Low: Mildly offensive or rude, not intended to cause serious harm or distress. \n
    - Medium: More offensive and potentially hurtful, but not extremely damaging. \n
    - High: Very offensive, aggressive, and intended to cause serious harm or distress. This includes racial, ethnic, gender-based, or any form of hate speech. \n

    Examples:\n
    - "You are a wonderful person." -> No \n
    - "I hate you and you are stupid." -> Yes, High \n
    - "You did a great job on the project." -> No \n
    - "You are so dumb." -> Yes, Medium \n
    - "Shut Up" -> No \n
    - "Sit Down" -> No \n
    - "You are such an idiot." -> Yes, Medium \n
    - "Get lost, nobody likes you." -> Yes, High \n
    - "Please be quiet." -> No \n
    - "Stop talking." -> No \n
    - "I said shut up!" -> No \n
    - "Suck it!" -> Yes, High \n
    - "You are immigrants." -> Yes, High \n
    - "Go back to your country." -> Yes, High \n
    

    Please respond to the following sentence. Answer in two words, Yes or Not and Levels only if first word is Yes else only one word answer."""

prompt_template = lambda comment: f'''[INST] {instruction_string} \n {comment} \n [\INST]'''

prompt = prompt_template(comment)

In [27]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Determine if the following sentence contains abusive language. If it does, specify the level of abusiveness as Low, Medium, or High. 

    
    Definitions: 

    - Low: Mildly offensive or rude, not intended to cause serious harm or distress. 

    - Medium: More offensive and potentially hurtful, but not extremely damaging. 

    - High: Very offensive, aggressive, and intended to cause serious harm or distress. This includes racial, ethnic, gender-based, or any form of hate speech. 


    Examples:

    - "You are a wonderful person." -> No 

    - "I hate you and you are stupid." -> Yes, High 

    - "You did a great job on the project." -> No 

    - "You are so dumb." -> Yes, Medium 

    - "Shut Up" -> No 

    - "Sit Down" -> No 

    - "You are such an idiot." -> Yes, Medium 

    - "Get lost, nobody likes you." -> Yes, High 

    - "Please be quiet." -> No 

    - "Stop talking." -> No 

    - "I said shut up!" -> No 

    - "Suck it!" -> Yes, High 

    - "You are

## Fine Tuning the model usign QLora for the better answers.

### Prepare the model for the training

In [36]:
from peft import prepare_model_for_kbit_training

model.train()  # In training mode, backprop and dropouts activated

model.gradient_checkpointing_enable()  # Kind of memory-saving techniques, holds the activation outputs and uses in backpropagation

model = prepare_model_for_kbit_training(model)  # Training in higher precision


## Setting up the LoRA for model parameter reduction

In [37]:
config = LoraConfig(
        r = 8,
        lora_alpha=32,
        target_modules=["q_proj"],  # Targeting the query modules in the model
        lora_dropout=0.05,
        bias = "none",
        task_type="CAUSAL_LM"
    
)

# LORA trainable version of model
model = get_peft_model(model,config)

model.print_trainable_parameters()


trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


## Loading the dataset"

In [38]:
data = load_dataset("shawhin/shawgpt-youtube-comments")

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9 [00:00<?, ? examples/s]

In [39]:
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})

In [41]:
data['train']['example'][0]

"<s>[INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.\n\nPlease respond to the following comment.\n \nThis is a about as perfect a coverage of this topic as I could imagine. I'm a researcher with a PhD in NLP who trains LLMs from scratch for a living and often find myself in need of communicating the process in a way that's digestible to a broad audience without back and forth question answering, so I'm thrilled to have found your piece! As an aside, I think the token order on the y-axis of the attention mask for decoders on slide 10 is reversed \n[/INST]\nThanks Sean! It's always a challe

## Tokenization Process

In [42]:
def tokenize_function(example):
    text = example["example"]
    
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )
    return tokenized_inputs

In [43]:
tokenized_data = data.map(tokenize_function,batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

## Data Collator

If examples are too long, so to do matrix multiplication, batches should have same size. Multiple length examples of different length will dynamically pad each examples so they have same length

In [45]:
tokenizer.pad_token = tokenizer.eos_token

data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

## Training Process

In [46]:
# Hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# Training Args

training_args = transformers.TrainingArguments(

    output_dir="Mistral7b",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",              # Every Epoch will print the training loss, evaluatoin loss
    evaluation_strategy="epoch",
    save_strategy="epoch",             # Save the model at each epoch
    load_best_model_at_end=True,       # Save the best performing model at epoch
    gradient_accumulation_steps=4,     
    warmup_steps=2,
    fp16=True,                            # 16 bit values for training
    optim="paged_adamw_8bit"
)

C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
# Configure the trainer

trainer = transformers.Trainer(
            model = model,
            train_dataset=tokenized_data['train'],
            eval_dataset = tokenized_data['test'],
            args = training_args,
            data_collator = data_collator

)

Using auto half precision backend


In [48]:
model.config.use_cache = False # silence the warnings

In [49]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: example. If example are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 50
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 30
  Number of trainable parameters = 2,097,152
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: dhakalrabin04 (ashrot07). Use `wandb login --relogin` to force relogin


C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,4.591000,3.957503
1,4.031900,3.416744
2,3.440000,2.946997
4,2.611800,2.252372
5,2.284100,2.061086
6,2.030800,1.891245
8,1.798000,1.728735
9,1.255900,1.723810


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: example. If example are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9
  Batch size = 4
Saving model checkpoint to Mistral7b\checkpoint-3
C:\Users\vinci\miniconda3\envs\vinci\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\vinci\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.2-GPTQ\snapshots\7532d6bc89ef9300fb39d2d94ed4414ec534b72a\config.json
Model config MistralConfig {
  "_name_or_path": "/workspace/process/mistralai_mistral-7b-instruct-v0.2/source",
  "architectures": [
   

TrainOutput(global_step=30, training_loss=2.6066402037938436, metrics={'train_runtime': 290.661, 'train_samples_per_second': 1.72, 'train_steps_per_second': 0.103, 'total_flos': 100119777067008.0, 'train_loss': 2.6066402037938436, 'epoch': 9.23076923076923})

## Model Evaluating / Inferencing

In [4]:
from peft import PeftConfig,PeftModel

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"  # THis guy has many quantized models
model = AutoModelForCausalLM.from_pretrained( # using pretrained method we can import modle
            model_name,   
            device_map = "cuda", # Optimum way to split the loading cpu and gpu working
            trust_remote_code=False, #not allow custom model file to run on local machine while downloading
            revision='main' # we want the main version

)

config = PeftConfig.from_pretrained("Mistral7b/checkpoint-30")
model = PeftModel.from_pretrained(model, "Mistral7b/checkpoint-30")

tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

In [5]:
intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "Great content, thank you!"

prompt = prompt_template(comment)
print(prompt)

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0]) 



comment = "What is fat-tailedness?"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280) 
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Great content, thank you! 
[/INST]

Glad you found it helpful! –ShawGPT

I'd be happy to answer any questions you have in the comments below. –ShawGPT

I'm glad you enjoyed the video! Let me know if you have any questions. –ShawGPT

I'm glad you found the video helpful! Let me know if you have any questions or if there's anything else I can help you with. –ShawGPT

I'm glad you found the video helpful! Let me know if you have any questions or if there's anything else I can help you with. I'll 